**Imports, defining variables, switching to GPU if available**

In [17]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms


batch_size = 64
num_classes = 10
learning_rate = 0.001
num_epochs = 20 


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

**Uploading and preprocessing data**

In [18]:
# converts images into normalized tensors
all_transforms = transforms.Compose([transforms.Resize((32,32)),
                                     transforms.ToTensor(),
                                     transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], #mean and std of each band in images can be found online 
                                                          std=[0.2023, 0.1994, 0.2010])
                                     ])
# Create Training dataset (using CIFAR-10, 60,000 RGB images, 10 different classes)
train_dataset = torchvision.datasets.CIFAR10(root = './data',
                                             train = True,
                                             transform = all_transforms,
                                             download = True)

# Create Testing dataset
test_dataset = torchvision.datasets.CIFAR10(root = './data',
                                            train = False,
                                            transform = all_transforms,
                                            download=True)

# Instantiate loader objects to facilitate processing
train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                           batch_size = batch_size,
                                           shuffle = True)


test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                           batch_size = batch_size,
                                           shuffle = True)

Files already downloaded and verified
Files already downloaded and verified


**Defining the CNN**

In [19]:
# Creating a CNN class
class ConvNeuralNet(nn.Module):

	#  Determine what layers and their order in CNN object 
    def __init__(self, num_classes):
        super(ConvNeuralNet, self).__init__()
        self.conv_layer1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3) #  Convolution layer #1 . Applies filter 
        self.conv_layer2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3) #  Convolution layer #2 
        self.max_pool1 = nn.MaxPool2d(kernel_size = 2, stride = 2) #  Pooling layer #1 (Max pooling) 
        
        self.conv_layer3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3) #Convolution layer 3
        self.conv_layer4 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3) #Convolution layer 4
        self.max_pool2 = nn.MaxPool2d(kernel_size = 2, stride = 2) #Pooling layer 2
        
        self.fc1 = nn.Linear(1600, 128) 
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(128, num_classes)
    
    # Progresses data across layers    
    def forward(self, x):
        out = self.conv_layer1(x)
        out = self.conv_layer2(out)
        out = self.max_pool1(out)
        
        out = self.conv_layer3(out)
        out = self.conv_layer4(out)
        out = self.max_pool2(out)
                
        out = out.reshape(out.size(0), -1)
        
        out = self.fc1(out)
        out = self.relu1(out)
        out = self.fc2(out)
        return out

**Model instantiation**

In [20]:
model = ConvNeuralNet(num_classes)

# Set Loss function with criterion
criterion = nn.CrossEntropyLoss()

# Set optimizer with optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay = 0.005, momentum = 0.9)  

total_step = len(train_loader)

**Training the model**

In [21]:
# We use the pre-defined number of epochs to determine how many iterations to train the network on
for epoch in range(num_epochs):
	#Load in the data in batches using the train_loader object
    for i, (images, labels) in enumerate(train_loader):  
        # Move tensors to the configured device
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

            
            


Epoch [1/20], Loss: 2.1950
Epoch [2/20], Loss: 1.2035
Epoch [3/20], Loss: 1.1720
Epoch [4/20], Loss: 0.9710
Epoch [5/20], Loss: 1.3299
Epoch [6/20], Loss: 1.1955
Epoch [7/20], Loss: 1.0788
Epoch [8/20], Loss: 1.3297
Epoch [9/20], Loss: 1.1859
Epoch [10/20], Loss: 0.4788
Epoch [11/20], Loss: 1.0794
Epoch [12/20], Loss: 0.6331
Epoch [13/20], Loss: 0.7212
Epoch [14/20], Loss: 0.4706
Epoch [15/20], Loss: 0.3120
Epoch [16/20], Loss: 0.3844
Epoch [17/20], Loss: 0.5744
Epoch [18/20], Loss: 0.5269
Epoch [19/20], Loss: 0.6438
Epoch [20/20], Loss: 0.5180


**Testing the model**

In [22]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    print('Accuracy of the network on the {} train images: {} %'.format(50000, 100 * correct / total))

Accuracy of the network on the 50000 train images: 82.794 %
